In [22]:
import requests
import json
import time
from dataclasses import dataclass
from pyspark.sql import SparkSession

In [24]:
#Test PySpark to Dataframe
colums = ["id", "name", "latitude", "longitude", "bikes", "free", "timestamp"]
data = [[1, "Test", 1.0, 1.0, 1, 1, 1]]
spark = SparkSession.builder.appName("Python Spark SQL basic example").getOrCreate()
rdd = spark.sparkContext.parallelize(data)

df = spark.createDataFrame(rdd, colums)
df.show()

RuntimeError: Java gateway process exited before sending its port number

### Get all Data with one request

In [4]:
import time


# while True:
resp = requests.get("https://iot.hamburg.de/v1.1/Things?$filter=Datastreams/properties/serviceName%20eq%20%27HH_STA_AutomatisierteVerkehrsmengenerfassung%27&$count=true&$expand=Locations,Datastreams($expand=Observations($top=3;$orderby=phenomenonTime%20desc),Sensor,ObservedProperty)")

response = json.loads(resp.content)
if resp.status_code == 200:
    print("OK")
    for feature in response['value']:
        print(feature['@iot.id'])
        for stream in feature['Datastreams']:
            if "Kfz" not in stream['description']:
                _ = print(stream['description'])
                _ = print(stream['Observations'][0]['result'])

        print("---------------------")
        
        # break
else:
    print("NOT OK")

    # time.sleep(300)
# print(ids, len(ids))


OK
5564
Die Anzahl der vom Infrarotdetektor erfassten Fahrräder wird für ein 1-Woche-Intervall aufsummiert.
38875
Die Anzahl der vom Infrarotdetektor erfassten Fahrräder wird für ein 15-Min-Intervall aufsummiert.
42
Die Anzahl der vom Infrarotdetektor erfassten Fahrräder wird für ein 1-Stunde-Intervall aufsummiert.
167
Die Anzahl der vom Infrarotdetektor erfassten Fahrräder wird für ein 1-Tag-Intervall aufsummiert.
4968
---------------------
5565
Die Anzahl der vom Infrarotdetektor erfassten Fahrräder wird für ein 15-Min-Intervall aufsummiert.
27
Die Anzahl der vom Infrarotdetektor erfassten Fahrräder wird für ein 1-Stunde-Intervall aufsummiert.
107
Die Anzahl der vom Infrarotdetektor erfassten Fahrräder wird für ein 1-Woche-Intervall aufsummiert.
21005
Die Anzahl der vom Infrarotdetektor erfassten Fahrräder wird für ein 1-Tag-Intervall aufsummiert.
2827
---------------------
5566
Die Anzahl der vom Infrarotdetektor erfassten Fahrräder wird für ein 15-Min-Intervall aufsummiert.
15
Die 

### Just 15 Min Data

In [20]:
@dataclass
class BikeSensor15Min:
    id: int
    name: str
    count: int
    time: str
    coordinateType: str
    coordinates: list

# class BikeSensor15Min:

# while True:
resp = requests.get("https://iot.hamburg.de/v1.0/Things?$filter=Datastreams/properties/serviceName%20eq%20%27HH_STA_HamburgerRadzaehlnetz%27%20and%20Datastreams/properties/layerName%20eq%20%27Anzahl_Fahrraeder_Zaehlfeld_15-Min%27&$count=true&$expand=Datastreams($filter=properties/layerName%20eq%20%27Anzahl_Fahrraeder_Zaehlfeld_15-Min%27;$expand=Observations($top=1;$orderby=phenomenonTime%20desc))")
response = json.loads(resp.content)
done = False
cnt = 0
sensors = []
while(not done):
    if resp.status_code == 200:
        for feature in response['value']:
            for stream in feature['Datastreams']:
                if len(stream['Observations']) > 0:
                    print(stream['description'])
                    print(stream['Observations'][0]['result'])
                    print(stream['Observations'][0]['phenomenonTime'])
                    bikesensor = BikeSensor15Min(feature['@iot.id'], feature['name'], stream['Observations'][0]['result'], stream['Observations'][0]['phenomenonTime'], stream['observedArea']['type'], stream['observedArea']['coordinates'])
                    sensors.append(bikesensor)
                else:
                    print(stream['description'])
                    print(0)
                    bikesensor = BikeSensor15Min(feature['@iot.id'], feature['name'], 0, stream['Observations'][0]['phenomenonTime'], stream['observedArea']['type'], stream['observedArea']['coordinates'])
                    sensors.append(bikesensor)
            print("---------------------")

            cnt += 1
    else:
        print("NOT OK")
    if '@iot.nextLink' not in response:
        done = True
    else:
        resp = requests.get(response['@iot.nextLink'])
        response = json.loads(resp.content)



print(cnt)
    # time.sleep(300)
# print(ids, len(ids))


Die Anzahl der von der Infrarotdetektor erfassten Fahrräder wird für ein 15-Min-Intervall aufsummiert.
9
2022-12-10T11:00:00.000Z/2022-12-10T11:14:59.000Z
---------------------
Die Anzahl der von der Infrarotdetektor erfassten Fahrräder wird für ein 15-Min-Intervall aufsummiert.
9
2022-12-10T11:00:00.000Z/2022-12-10T11:14:59.000Z
---------------------
Die Anzahl der von der Infrarotdetektor erfassten Fahrräder wird für ein 15-Min-Intervall aufsummiert.
4
2022-12-10T11:00:00.000Z/2022-12-10T11:14:59.000Z
---------------------
Die Anzahl der von der Infrarotdetektor erfassten Fahrräder wird für ein 15-Min-Intervall aufsummiert.
1
2022-12-10T11:00:00.000Z/2022-12-10T11:14:59.000Z
---------------------
Die Anzahl der von der Infrarotdetektor erfassten Fahrräder wird für ein 15-Min-Intervall aufsummiert.
0
---------------------
Die Anzahl der von der Infrarotdetektor erfassten Fahrräder wird für ein 15-Min-Intervall aufsummiert.
0
---------------------
Die Anzahl der von der Infrarotdetekto

### Just 1 Hour Data

In [28]:
import time


# while True:
resp = requests.get("https://iot.hamburg.de/v1.0/Things?$filter=Datastreams/properties/serviceName%20eq%20%27HH_STA_HamburgerRadzaehlnetz%27%20and%20Datastreams/properties/layerName%20eq%20%27Anzahl_Fahrraeder_Zaehlstelle_1-Stunde%27&$count=true&$expand=Datastreams($filter=properties/layerName%20eq%20%27Anzahl_Fahrraeder_Zaehlstelle_1-Stunde%27;$expand=Observations($top=1;$orderby=phenomenonTime%20desc))")
response = json.loads(resp.content)
if resp.status_code == 200:
    print("OK")
    for feature in response['value']:
        print(feature['@iot.id'])
        for stream in feature['Datastreams']:
            if len(stream['Observations']) > 0:
                print(stream['description'])
                print(stream['Observations'][0]['result'])
        
        # break
else:
    print("NOT OK")

    # time.sleep(300)
# print(ids, len(ids))


OK
5564
Die Anzahl der vom Infrarotdetektor erfassten Fahrräder wird für ein 1-Stunde-Intervall aufsummiert.
193
5565
Die Anzahl der vom Infrarotdetektor erfassten Fahrräder wird für ein 1-Stunde-Intervall aufsummiert.
61
5566
Die Anzahl der vom Infrarotdetektor erfassten Fahrräder wird für ein 1-Stunde-Intervall aufsummiert.
132
5570
Die Anzahl der vom Infrarotdetektor erfassten Fahrräder wird für ein 1-Stunde-Intervall aufsummiert.
74
5571
Die Anzahl der vom Infrarotdetektor erfassten Fahrräder wird für ein 1-Stunde-Intervall aufsummiert.
26
5572
Die Anzahl der vom Infrarotdetektor erfassten Fahrräder wird für ein 1-Stunde-Intervall aufsummiert.
48
5573
Die Anzahl der vom Infrarotdetektor erfassten Fahrräder wird für ein 1-Stunde-Intervall aufsummiert.
29
5574
Die Anzahl der vom Infrarotdetektor erfassten Fahrräder wird für ein 1-Stunde-Intervall aufsummiert.
3
5575
Die Anzahl der vom Infrarotdetektor erfassten Fahrräder wird für ein 1-Stunde-Intervall aufsummiert.
5
5576
Die Anzahl 